In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

In [2]:
DIR = "../../data/"
SMOOTHIE = "Smoothie King/"

smoothie_demographic = pd.read_csv(DIR + SMOOTHIE + "processed_demographic.csv")
smoothie_stores = pd.read_csv(DIR + SMOOTHIE + "smoothie_king_stores.csv")
smoothie_poi_variables = pd.read_csv(DIR + SMOOTHIE + "processed_poi.csv")
# smoothie_sister = pd.read_csv(DIR + SMOOTHIE + "competition_sister_variables.csv")
smoothie_trade_area = pd.read_csv(DIR + SMOOTHIE + "processed_trade_area.csv").rename(columns={"store_num": "store"})

In [3]:
smoothie_merged = smoothie_stores.merge(
    smoothie_demographic, on="store", how="outer"
).merge(
    smoothie_poi_variables, on="store", how="outer"
).merge(
    smoothie_trade_area, on="store", how="outer"
)
smoothie_merged

,store,longitude,latitude,category,cbsa_name,dma_name,state_name,market_size,store_density,age0018_p_10mi,...,popgrfy_ta,popgrpy_ta,poverty_inpoverty_p_ta,spend_breakfastbrunch_ta,spend_dinner_ta,spend_foodbev_ta,spend_lunch_ta,wealth_hhavg_ta,wealth_hhtotal_ta,white_p_ta
0,SK 1504,-97.650392,30.519353,SHOPPING,"Austin-Round Rock, TX","Austin, TX",Texas,Large Metro (2),Light Suburban,0.2805,...,8.3789,3.9235,0.0611,7069439.0,40790484.0,230383651.0,23166216.0,240573.0,25223.0,0.4897
1,SK 0057,-88.171150,30.672501,SHOPPING,"Mobile, AL","Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,0.2264,...,0.6017,0.3932,0.1830,4724526.0,25460067.0,160135521.0,14653701.0,217054.0,22216.0,0.5129
2,SK 1415,-90.535722,38.784250,HOME,"St. Louis, MO-IL","St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,0.2129,...,2.5003,0.7142,0.0639,4501211.0,24794631.0,151609187.0,14129014.0,245860.0,19907.0,0.8459
3,SK 1231,-80.134700,26.100737,TRAVEL,"Miami-Fort Lauderdale et al, FL","Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,0.2174,...,7.6482,12.7188,0.0831,4038906.0,23214366.0,146417939.0,12859709.0,255812.0,22124.0,0.7020
4,SK 1535,-96.856651,32.996408,WORK,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,0.2620,...,2.1871,-5.7794,0.0800,5919218.0,32751951.0,207616741.0,18696840.0,216763.0,29251.0,0.4593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,-96.872596,32.647809,HOME,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,0.2664,...,2.0540,-0.2994,0.1833,7614977.0,40037412.0,264008408.0,23332500.0,199437.0,29430.0,0.1490
792,SK 1886,-105.077634,40.564695,OTHER,"Fort Collins, CO","Denver, CO",Colorado,Medium City (4),Light Suburban,0.2224,...,4.7691,2.3442,0.2202,5118356.0,28458285.0,197532602.0,16637181.0,216659.0,25712.0,0.7874
793,SK 0162,-95.478001,30.316531,SHOPPING,"Houston-The Woodlands et al, TX","Houston, TX",Texas,Very Large Metro (1),Exurban,0.2412,...,9.9822,8.1691,0.1202,4677280.0,25445615.0,161088758.0,14627532.0,216855.0,19407.0,0.5041
794,SK 1449,-78.968258,35.064994,SHOPPING,"Fayetteville, NC","Raleigh et al, NC",North Carolina,Medium City (4),Exurban,0.2614,...,-0.3816,-1.0761,0.1809,10318478.0,54577513.0,349542567.0,31683602.0,208448.0,41681.0,0.3933


In [4]:
train_df, test_df = train_test_split(smoothie_merged, test_size=0.1, random_state=42)
X_train = train_df.drop(columns=["category"])
y_train = train_df["category"]
X_test = test_df.drop(columns=["category"])
y_test = test_df["category"]

In [5]:
drop_features = ['store',
 'longitude',
 'latitude',
 'cbsa_name',
 'dma_name',
 'state_name',
 ]

passthrough_features = []

ordinal_features_oth = [
    "market_size",
    "store_density",
]
ordering_ordinal_oth = [
    ["Very Large Metro (1)", "Large Metro (2)", "Large City (3)", "Medium City (4)", "Small City (5)", "Small Town (6)"],
    ["Rural", "Exurban", "Suburban", "Light Suburban", "Light Urban", "Urban", "Super Urban"],
]
numeric_features = list(set(smoothie_merged.select_dtypes(include=np.number).columns.tolist()) - {"longitude", "latitude"})

In [6]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy="median"), 
    StandardScaler()
)

ordinal_transformer_oth = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(categories=ordering_ordinal_oth),
)

preprocessor = make_column_transformer(
    ("drop", drop_features),
    (numeric_transformer, numeric_features),
    (ordinal_transformer_oth, ordinal_features_oth),
    ("passthrough", passthrough_features),
)

In [7]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('drop', 'drop',
                                 ['store', 'longitude', 'latitude', 'cbsa_name',
                                  'dma_name', 'state_name']),
                                ('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['emp_utilities_p_10mi', 'emp_utilities_p_3mi',
                                  'inrix_dinner_ew',
                                  'ipeds_postsecondary_schools_total_enrollment_10mi...
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Very '
                                                                              'Large '
                                                                              'Metro '
                                                                              '(1)',
                                                                              'Large '
                                                                              'Metro '
                                                                              '(2)',
                                                                              'Large '
                                                                              'City '
                                                                              '(3)',
                                                                              'Medium '
                                                                              'City '
                                                                              '(4)',
                                                                              'Small '
                                                                              'City '
                                                                              '(5)',
                                                                              'Small '
                                                                              'Town '
                                                                              '(6)'],
                                                                             ['Rural',
                                                                              'Exurban',
                                                                              'Suburban',
                                                                              'Light '
                                                                              'Suburban',
                                                                              'Light '
                                                                              'Urban',
                                                                              'Urban',
                                                                              'Super '
                                                                              'Urban']]))]),
                                 ['market_size', 'store_density']),
                                ('passthrough', 'passthrough', [])])

In [8]:
column_names = (
    numeric_features
    + preprocessor.named_transformers_['pipeline-2']['ordinalencoder'].get_feature_names_out().tolist()
)
len(column_names)

920

In [9]:
transformed_X_train = preprocessor.transform(X_train)

In [10]:
transformed_X_train_df = pd.DataFrame(transformed_X_train, columns=column_names)
transformed_X_train_df

,emp_utilities_p_10mi,emp_utilities_p_3mi,inrix_dinner_ew,ipeds_postsecondary_schools_total_enrollment_10mi,edu_bachplus_female_p_ta,genx_p_3mi,emp_healthcare_social_assistance_p_1mi,emp_construction_p_5mi,hh_0vehicle_p_10mi,mortgage_avgrisk_2mi,...,occ_military_p_1mi,emp_wholesale_trade_p_2mi,emp_other_p_10mi,emp_transportation_storage_p_5mi,edu_bachplus_female_p_3mi,emp_construction_p_ta,popcy_ta,hh_1vehicle_p_10mi,x0,x1
0,-0.560634,-0.507976,0.546785,-0.197722,0.931239,1.240721,-0.768309,1.914418,-1.000214,0.357442,...,-0.073391,-0.543532,-0.769315,-0.073795,1.144127,1.382958,2.601410,-1.774350,1.0,0.0
1,-0.645481,0.096195,1.066532,-0.059481,1.133256,1.116726,1.057724,-0.832126,0.989572,-0.659683,...,-0.073391,0.442923,-0.322498,-0.580823,1.285841,-0.324496,1.148569,0.712010,0.0,3.0
2,0.146429,-0.463768,-0.980738,-0.756806,0.849086,0.206140,2.469029,-0.751213,-1.027139,-0.514516,...,-0.073391,-0.740823,0.045470,-0.625040,1.096889,0.201679,-0.770199,-2.035976,0.0,1.0
3,1.899946,-0.080636,0.964912,0.331209,-1.316530,-0.258840,-0.456304,3.460754,-0.453629,-1.036479,...,-0.073391,0.233302,0.273259,0.542303,-1.323698,3.989440,0.074142,-0.453414,0.0,3.0
4,0.259559,-0.493240,1.080124,-0.418480,0.206674,-0.960185,-0.328282,-0.611863,-0.138602,0.299695,...,-0.073391,-0.605186,-0.760554,-0.501231,0.305295,-0.118905,0.268223,-0.956541,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,-0.645481,-0.404825,-0.062282,0.353120,2.232225,1.767699,1.850385,0.718705,0.165655,0.389028,...,-0.073391,-0.432556,0.536093,0.070649,1.489107,0.072748,-1.003930,1.077920,0.0,2.0
712,0.287842,0.169874,-1.011806,-0.493216,0.734610,0.237139,1.587443,-0.472513,0.216813,0.306714,...,-0.073391,-0.654508,1.035477,-0.521866,0.686062,-0.327981,0.377865,0.477827,3.0,1.0
713,-0.730329,-0.404825,1.782396,0.083128,-0.253924,0.806740,-1.092579,-0.162347,-1.191384,0.109861,...,-0.073391,-0.161281,-0.278692,-0.492388,-0.151338,0.128502,0.239418,-1.785327,0.0,1.0
714,-0.390939,-0.286938,-0.369081,-0.479439,0.556836,-0.010851,-1.155440,-1.007438,-0.222070,-0.702755,...,-0.073391,-0.777815,0.141842,-0.645675,0.371142,-0.913394,-1.325745,0.496123,2.0,1.0


In [11]:
import optuna
from optuna.integration import LightGBMPruningCallback

In [12]:
best_booster = None
lgbm = None

In [13]:
def objective(trial, X, y):
    global lgbm
    param_grid = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [200]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 400, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 500, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        # print(idx)
        # print(train_idx)
        # print(test_idx)
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        # print('X_train shape: ', X_train.shape)
        # print('X_test shape: ', X_test.shape)
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        # print('y_train shape: ', y_train.shape)
        # print('y_test shape: ', y_test.shape)

        lgbm = LGBMClassifier(objective="multiclass", **param_grid)
        lgbm.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "multi_logloss")
            ],  # Add a pruning callback
        )
        preds = lgbm.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [14]:
def callback(study, trial):
    global best_booster
    if study.best_trial == trial:
        best_booster = lgbm

In [15]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, transformed_X_train_df, y_train)
study.optimize(func, n_trials=100, callbacks=[callback])

[I 2023-05-17 14:04:50,267] A new study created in memory with name: LGBM Classifier


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=65, reg_lambda=0.0 will be ignored. Current value: lambda_l2=65
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] min_gain_to_split is set=4.406424547902308, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.406424547902308
[LightGBM] [Warning] lambda_l1 is set=35, reg_alpha=0.0 will be ignored. Current value: lambda_l1=35
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samp

[I 2023-05-17 14:04:50,635] Trial 0 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.06978432189209867, 'num_leaves': 60, 'max_depth': 12, 'min_data_in_leaf': 500, 'max_bin': 241, 'lambda_l1': 35, 'lambda_l2': 65, 'min_gain_to_split': 4.406424547902308, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 1.4718720941213552.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] lambda_l2 is set=65, reg_lambda=0.0 will be ignored. Current value: lambda_l2=65
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] min_gain_to_split is set=4.406424547902308, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.406424547902308
[LightGBM] [Warning] lambda_l1 is set=35, reg_alpha=0.0 will be ignored. Current value: lambda_l1=35
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samp

[I 2023-05-17 14:04:50,961] Trial 1 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.22972370502866815, 'num_leaves': 20, 'max_depth': 6, 'min_data_in_leaf': 300, 'max_bin': 252, 'lambda_l1': 20, 'lambda_l2': 75, 'min_gain_to_split': 1.3548661861908806, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 0 with value: 1.4718720941213552.


...

[I 2023-05-17 14:05:29,340] Trial 97 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.09924489208777909, 'num_leaves': 360, 'max_depth': 10, 'min_data_in_leaf': 300, 'max_bin': 208, 'lambda_l1': 20, 'lambda_l2': 70, 'min_gain_to_split': 5.126910764571017, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 1.4718720941213552.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] lambda_l2 is set=70, reg_lambda=0.0 will be ignored. Current value: lambda_l2=70
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] min_gain_to_split is set=5.126910764571017, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.126910764571017
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samp

[I 2023-05-17 14:05:29,704] Trial 98 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.17921520609443742, 'num_leaves': 40, 'max_depth': 4, 'min_data_in_leaf': 400, 'max_bin': 228, 'lambda_l1': 60, 'lambda_l2': 65, 'min_gain_to_split': 14.93128783345878, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 0 with value: 1.4718720941213552.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] lambda_l2 is set=65, reg_lambda=0.0 will be ignored. Current value: lambda_l2=65
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_gain_to_split is set=14.93128783345878, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=14.93128783345878
[LightGBM] [Warning] lambda_l1 is set=60, reg_alpha=0.0 will be ignored. Current value: lambda_l1=60
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samp

[I 2023-05-17 14:05:30,109] Trial 99 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.08105039712736806, 'num_leaves': 340, 'max_depth': 3, 'min_data_in_leaf': 400, 'max_bin': 249, 'lambda_l1': 50, 'lambda_l2': 20, 'min_gain_to_split': 0.0502058941699598, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 1.4718720941213552.


In [16]:
print(f"\tBest value (multi_logloss): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (multi_logloss): 1.47187
	Best params:
		n_estimators: 200
		learning_rate: 0.06978432189209867
		num_leaves: 60
		max_depth: 12
		min_data_in_leaf: 500
		max_bin: 241
		lambda_l1: 35
		lambda_l2: 65
		min_gain_to_split: 4.406424547902308
		bagging_fraction: 0.4
		bagging_freq: 1
		feature_fraction: 0.4


In [17]:
best_booster

LGBMClassifier(bagging_fraction=0.4, bagging_freq=1, feature_fraction=0.4,
               lambda_l1=35, lambda_l2=65, learning_rate=0.06978432189209867,
               max_bin=241, max_depth=12, min_data_in_leaf=500,
               min_gain_to_split=4.406424547902308, n_estimators=200,
               num_leaves=60, objective='multiclass')

In [18]:
# from sklearn.preprocessing import LabelEncoder

In [19]:
# le = LabelEncoder()
# smoothie_merged["category"] = le.fit_transform(smoothie_merged["category"])

In [20]:
class_weight = {
    "HOME": 0.31,
    "OTHER": 0.13,
    "SHOPPING": 0.22,
    "TRAVEL": 0.15,
    "WORK": 0.28
}
# encoded_class_weights = {i: class_weight[label] for i, label in enumerate(le.classes_)}
# class_weight = encoded_class_weights

In [21]:
pipe_rf = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_jobs=-1, random_state=42)
)
pipe_rf.fit(X_train, y_train);

In [22]:
rf_param_grid = {
    "randomforestclassifier__n_estimators": [25, 50, 100],
    "randomforestclassifier__max_features": ["sqrt", "log2", None],
    "randomforestclassifier__max_depth": [5, 10, 20, 50],
    "randomforestclassifier__max_leaf_nodes": [30, 50, 70],
    "randomforestclassifier__min_samples_split": [20, 25, 30, 35, 40, 45, 50],
    "randomforestclassifier__min_samples_leaf": [30, 50, 70],
    "randomforestclassifier__class_weight": [None, "balanced"]
}
rf_grid_search = GridSearchCV(
    pipe_rf, rf_param_grid, cv=5, n_jobs=-1, return_train_score=True
)
rf_grid_search.fit(X_train, y_train);

In [23]:
rf_tuned = rf_grid_search.best_estimator_
best_rf = rf_tuned.named_steps["randomforestclassifier"]
type(best_rf)
best_rf

RandomForestClassifier(class_weight='balanced', max_depth=20, max_features=None,
                       max_leaf_nodes=30, n_jobs=-1, random_state=42)

In [24]:
def evaluate_models(X_train, y_train,
		            X_test, y_test,
		    		rfe_estimator, 
		    		rfe_num_features_start, rfe_num_features_end, rfe_num_features_step, 
				    pipe_model):
	
	for i in range(rfe_num_features_start, rfe_num_features_end+1, rfe_num_features_step):
		print('---- RFE with ', i, ' features selected ----')
		
		rfe = RFE(estimator=rfe_estimator, n_features_to_select=i)
		pipe_rfe_model = make_pipeline(preprocessor, 
			                           rfe, 
				                       pipe_model)
		
		# print(pipe_rfe_model)
		
		pipe_rfe_model.fit(X_train, pd.DataFrame(y_train))
		
		print(pipe_rfe_model.score(X_train, pd.DataFrame(y_train)))
		print(pipe_rfe_model.score(X_test, pd.DataFrame(y_test)))	

In [34]:
evaluate_models(X_train, y_train,
		        X_test, y_test,
		    	best_booster, 
		    	20, 35, 2, 
				RandomForestClassifier(random_state=42))

---- RFE with  20  features selected ----
1.0
0.3875
---- RFE with  22  features selected ----
1.0
0.4
---- RFE with  24  features selected ----
1.0
0.3875
---- RFE with  26  features selected ----
1.0
0.45
---- RFE with  28  features selected ----
1.0
0.4125
---- RFE with  30  features selected ----
1.0
0.3875
---- RFE with  32  features selected ----
1.0
0.3875
---- RFE with  34  features selected ----
1.0
0.35
